# Solve with Algorithm 3

To include Line numbers: select cell -> ctrl + m -> l (= ell) (without ctrl)

### Disitrubtion dependence 
The code as written below works for *all* instances. 

However for possible speed-up we've added minor changes for instances from the discrete Uniform distribution. Our changes can be found as comment in the code. These changes need not be made in order for the code to work on instances form the Uniform distribution. 


In [1]:
using JuMP
using CPLEX

# Algorithm 3

In [2]:
function Algorithm3(n::Int,d::Int,C::Array{Float32,3})
#function Algorithm3(n::Int,d::Int,m::Int,C::Array{Int64,3})
    
     mi = minimum(C);  # remove this line for Uniform specific
     ma = maximum(C);  # remove this line for Uniform specific
    
     OPT = ma+1;   # OPT = 2*m; 
     ArgOpt = [];  
    
# step 1: select x from the ground set; x=(p1,p2)
    x1 = 0; # translation from 2D array (p1,p2) to 1D array reference for OPt
    
    for p2 in 1: n
        for p1 in 1:n-1
            x1 = x1 + 1;   # translation from 2D array (p1,p2) to 1D array reference for OPt
            
# step 2: Set S_x = \emptyset; or here a list. 
            size = (n*(n-(p1-1)) - n -(n-p1));
            Delta = Float32[ma+1 for k in 1:size];  #Int[2*m for k in 1:size];
            
# step 3:  for each y 
            index = 1;
            
            for q2 in 1:n
                if(q2 != p2)
                    for q1 in p1+1:n

        # step 4: calculate \Delta_max(x,y)                  
                        upper = Float32[ma+1 for k in 1:d]; # upper = Float32[2*m for k in 1:d]
                        lower = Float32[mi-1 for k in 1:d]; # lower = Float32[0 for k in 1:d];
                        for k in 1:d
                            if(C[p1,p2,k] <= C[q1,q2,k])
                                lower[k] = C[p1,p2,k]
                                upper[k] = C[q1,q2,k]
                            else
                                upper[k] = C[p1,p2,k]
                                lower[k] = C[q1,q2,k]
                            end
                        end

                        Delta[index] = maximum(upper-lower);   
                        index = index+1;
                    end # for q1
                end #if q2 != p2
            end# for q2
            
           
# step 5: sort unique Delta values            
            Delta = unique(Delta)
            sort!(Delta) 

# step 6-14 : Binary search to find lowest Delta with feasible solution. 
            TempOpt = ma+1;  #TempOpt = 2*m;
            TempArgOpt = [];
            left = 1;  #left bound on range for search.
            right = length(Delta);  # right bound on range for search.
            middle = 0;
            FeasibleFound = false;   #if false, then increase left bound for larger Delta, if true then decrease right bound.
            EndWhile = false

            while (EndWhile==false) 
            
    # step 7: let an element from S_x be selected
                if(left != right)
                    middle = left + cld(right-left,2);  #cld is build-in ceil function of (right-left)/2
                   # println(left, "- ",middle, "-", right)
                else
                    middle = left;
                    EndWhile=true;
                    #println(left, "- ",middle, "-", right)
                end
             
    # step 8: Y := X  
                M = Bool[1 for i in 1:n, j in 1:n]; #  matrix for max weight Assignment Problem
    
    # step 9: remove elements if needed; i.e. set weight in matrix M to 0  
                for b in 1:n, a in 1:n 
                    if (M[a,b] == 1)
                        if(maximum(abs(C[a,b,k] - C[p1,p2,k]) for k in 1:d) > Delta[middle])
                              M[a,b] = 0     
                          end
                    end
                end # for a,b

                # If there exists any row, or column in which no 1 occurs; then we know it is infeasible
                # no model needs to be made and solved. 
                TestNeeded = true;
                for a in 1:n 
                    if( sum( M[a,x] for x in 1:n) < 0.9)
                        TestNeeded = false;
                    end
                end
                if(TestNeeded == true)
                    for b in 1:n 
                        if( sum( M[x,b] for x in 1:n) < 0.9)
                            TestNeeded = false;
                        end
                    end
                end

    # step 12: feasibility oracle; maximum weight assignment            
                if(TestNeeded == true)
                    
                    modelFeas = Model(solver=CplexSolver(CPX_PARAM_SCRIND=0));  
                    @variable(modelFeas,0 <= y[1:n,1:n] <= 1, Int);

                    @constraints(modelFeas, begin       
                        Assignment1[x1 in 1:n], sum(y[x1,x2] for x2 = 1:n) == 1
                        Assignment2[x2 in 1:n], sum(y[x1,x2] for x1 = 1:n) == 1
                    end);

                    @objective(modelFeas, Max, sum(y[x1,x2]*M[x1,x2] for x2 = 1:n, x1 in 1:n));
                    status = solve(modelFeas);
                    getObj = getobjectivevalue(modelFeas); 
                    
                    # if feasible, then compute Sol. 
                    if(getObj > n - 0.5) # allow for numerical inaccuracy
                        FeasibleFound = true;
                        
                        upper = Float32[mi-1 for k in 1:d]; # upper = Int[0 for k in 1:d];
                        lower = Float32[ma+1 for k in 1:d]; # lower = Int[2*m for k in 1:d];
                        
                        for(z2 in 1:n)
                            for(z1 in 1:n)
                                if(getvalue(y)[z1,z2] == 1)
                                    for k in 1:d
                                        if(C[z1,z2,k] <= lower[k])
                                            lower[k] = C[z1,z2,k]
                                        end
                                        if(C[z1,z2,k] >= upper[k])
                                            upper[k] = C[z1,z2,k]
                                        end
                                    end # end for k
                                end
                            end # for z1
                        end #for z2

                        Sol = maximum(upper-lower);
                        index = find(a->a==Sol, upper-lower)
                        if(OPT > Sol)
                            TempArgOpt = index;
                            TempOpt = Sol;
                        end
                        
                        end #if GetObj
                    end # if TestNeeded 

     #step 12-13: if Y contains feasible solution; select smaller element
                if( FeasibleFound == true ) 
                    right = middle - 1; 
                    else 
    # step 14: Else; select larger element
                    left = min( middle + 1, right) ; 
                    end 
                
                end # end while; the binary search. 
            
    # step 15: keep best solution found so far
                if(OPT > TempOpt)
                    ArgOpt = TempArgOpt;
                    OPT = TempOpt;
                end
            

            end # end for p1
        end # end for p2 
    print(" ",OPT, " ", sum(ArgOpt)/length(ArgOpt)," ", length(ArgOpt), " ");  
    
end

Algorithm3 (generic function with 1 method)

# choose instance type + run

First call of the function requires more running time, due to compiling.
Hence we first solve a small test instance, before running the actual instances.

In [ ]:
   println("testing + compiling")
    C3 = Float32[1 for i in 1:3, j in 1:3, k in 1:3];
    temp = readdlm(pwd()"/test/Norm1.txt")
    for(j in 1:3)
        for i in 1:3
          for k in 1:3
             C3[i,j,k] = temp[3*(i-1)+j,k];    
          end
        end
    end
    @time Algorithm3(3,3,C3)


#############  Run all instances Class 1  #########

for m in [100,1000]
    for n in [10,20]  
        println(" ")
        println(" ")
        for d in [2,100,300]
            println("ALG3:Uniform", m, "n=", n, "d=", d)
            for iter in 1:10
                #local C2 = Int64[1 for i in 1:n, j in 1:n, k in 1:d];
                local C2 = Float32[1 for i in 1:n, j in 1:n, k in 1:d];
                temp = readdlm(pwd()"/U(0,"*string(m)*")/n"*string(n)*"d"*string(d)*"/Ist"*string(iter)*".txt")    
                for( j in 1:n)
                    for i in 1:n
                        for k in 1:d
                            C2[i,j,k] = temp[n*(i-1)+j,k];    
                        end
                    end
                end
                print("Ist", iter)
               #@time Algorithm3(n,d,m,C2)
                @time Algorithm3(n,d,C2)
            end
        end
    end
end

for E in [50,500]
    for S in [1,10]
        println(" ")
        println(" ")
        for n in [10,20]
            for d in [2,100,300]
                println("ALG3:Normal(",E,",",S,")n=", n, "d=", d)
                for iter in 7:10

                    local C2 = Float32[1 for i in 1:n, j in 1:n, k in 1:d];
                    temp = readdlm(pwd()"/N("*string(E)*","*string(S)*")/n"*string(n)*"d"*string(d)*"/Ist"*string(iter)*".txt")
                    for( j in 1:n)
                        for i in 1:n
                            for k in 1:d
                                C2[i,j,k] = temp[n*(i-1)+j,k];    
                            end
                        end
                    end
                    print("Ist", iter)
                    @time Algorithm3(n,d,C2)

                end
            end
        end 
    end
end


testing + compiling
 1.1839523 3.0 1   4.651883 seconds (5.50 M allocations: 230.123 MB, 1.59% gc time)
 
 
ALG3:Uniform100n=10d=2
Ist1 35.0 2.0 1   0.530526 seconds (861.68 k allocations: 38.990 MB, 1.66% gc time)
Ist2 36.0 1.0 1   0.589303 seconds (894.37 k allocations: 42.385 MB, 0.89% gc time)
Ist3 40.0 1.0 1   0.523259 seconds (890.75 k allocations: 39.746 MB, 0.41% gc time)
Ist4 47.0 1.5 2   0.537897 seconds (863.39 k allocations: 39.545 MB, 0.78% gc time)
Ist5 42.0 1.0 1   0.529850 seconds (873.72 k allocations: 40.024 MB, 0.78% gc time)
Ist6 35.0 2.0 1   0.724420 seconds (894.72 k allocations: 41.929 MB, 1.24% gc time)
Ist7 38.0 2.0 1   0.614466 seconds (894.26 k allocations: 43.931 MB, 0.67% gc time)
Ist8 38.0 1.5 2   0.617244 seconds (898.44 k allocations: 43.256 MB, 0.68% gc time)
Ist9 40.0 1.0 1   0.551760 seconds (886.61 k allocations: 40.890 MB, 0.79% gc time)
Ist10 43.0 1.5 2   0.555449 seconds (884.22 k allocations: 40.889 MB, 0.77% gc time)
ALG3:Uniform100n=10d=100
Ist